In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [2]:
import re
import nltk
import string
from nltk.util import pr
stemmer=nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))

In [3]:
df = pd.read_csv('tweets.csv')
print(df)

       Unnamed: 0  class                                              tweet
0             0.0      2  !!! RT @mayasolovely: As a woman you shouldn't...
1             1.0      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2             2.0      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3             3.0      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4             4.0      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...           ...    ...                                                ...
25408     26160.0      2                          chut me lund  ghusa dunga
25409     26161.0      2                          chut me lund  ghusa dunga
25410     26162.0      2                         teri gand me bum dal dunga
25411     26163.0      1                        teri jaat ki gand maro bsdk
25412     26164.0      1                 tum chutiyo ki koi aukaat nahi hai

[25413 rows x 3 columns]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25413 entries, 0 to 25412
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  25412 non-null  float64
 1   class       25413 non-null  int64  
 2   tweet       25413 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 595.7+ KB


In [5]:
df['labels']=df['class'].map({0:"More offensive",1:"less offensive",2:"Nothing offensive"})
print(df.head())

   Unnamed: 0  class                                              tweet  \
0         0.0      2  !!! RT @mayasolovely: As a woman you shouldn't...   
1         1.0      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2         2.0      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3         3.0      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4         4.0      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

              labels  
0  Nothing offensive  
1     less offensive  
2     less offensive  
3     less offensive  
4     less offensive  


In [6]:
df=df[['tweet','labels']]
print(df)



                                                   tweet             labels
0      !!! RT @mayasolovely: As a woman you shouldn't...  Nothing offensive
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...     less offensive
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...     less offensive
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...     less offensive
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     less offensive
...                                                  ...                ...
25408                          chut me lund  ghusa dunga  Nothing offensive
25409                          chut me lund  ghusa dunga  Nothing offensive
25410                         teri gand me bum dal dunga  Nothing offensive
25411                        teri jaat ki gand maro bsdk     less offensive
25412                 tum chutiyo ki koi aukaat nahi hai     less offensive

[25413 rows x 2 columns]


In [7]:
df.sample(50)

,tweet,labels
24606,trash RT @VibeMagazine: Can the Notorious B.I....,Nothing offensive
24649,we live in a world where drake can steal your ...,less offensive
11822,"Instead of calling each other hoes, girls shou...",less offensive
398,"""@splurgb_: Nothin in life is free.. Some of y...",less offensive
21623,The two ringless fags RT @EverybodyHatesX: Bar...,Nothing offensive
6273,@joeylattime ur welcome u bitch,less offensive
8627,Damn internal alarm clock. Woke up at 6:35am f...,Nothing offensive
14688,RT @ChippewaSkylark: i love when celeb nudes l...,less offensive
23705,cant fuck around wit young niggas cause they b...,less offensive
12629,Lol.. shut up! RT @ABrown252: @VonshayeB lmao ...,Nothing offensive


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25413 entries, 0 to 25412
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   25413 non-null  object
 1   labels  25413 non-null  object
dtypes: object(2)
memory usage: 397.2+ KB


In [9]:
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=[word for word in text.split(' ')if word not in stopword]
    text=" ".join(text)
    text=[stemmer.stem(word)for word in text.split(' ')]
    text=" ".join(text)
    return text
df["tweet"]=df["tweet"].apply(clean)
print(df.head())


                                               tweet             labels
0   rt mayasolov woman shouldnt complain clean ho...  Nothing offensive
1   rt  boy dat coldtyga dwn bad cuffin dat hoe  ...     less offensive
2   rt urkindofbrand dawg rt  ever fuck bitch sta...     less offensive
3             rt cganderson vivabas look like tranni     less offensive
4   rt shenikarobert shit hear might true might f...     less offensive


In [10]:
x=np.array(df["tweet"])
y=np.array(df["labels"])
cv=CountVectorizer()
x=cv.fit_transform(x)
dictt = cv.get_feature_names() 
print(dictt)

print(len(dictt))




['aa', 'aaaaaaaaand', 'aaahhhhh', 'aag', 'aahahah', 'aaliyah', 'aaliyahhhm', 'aaliyahhlove', 'aamaadmiparti', 'aampt', 'aamu', 'aan', 'aap', 'aaron', 'aaroncart', 'aaronkeplin', 'aaronmacgrud', 'aaronthegoat', 'aaronworth', 'aaryn', 'aaryneliza', 'aarynelizagri', 'aasthaxoxo', 'ab', 'abandonado', 'abartick', 'abbey', 'abbeygoldd', 'abbeykirkland', 'abbeykuck', 'abbi', 'abc', 'abcrispyyandi', 'abdashcol', 'abdelka', 'abduct', 'abdullah', 'abdullahomar', 'abdurahman', 'abe', 'abel', 'abellaxxx', 'abelsmasochist', 'aberdeen', 'abil', 'abl', 'abo', 'abort', 'abortionfund', 'abound', 'abouta', 'aboutir', 'aboutstraf', 'abouttim', 'abraham', 'abrehamdadi', 'abreumegan', 'absent', 'absolut', 'absoluteyvil', 'absolv', 'absoulxo', 'abstract', 'abstractlif', 'abstraktalant', 'absurd', 'abt', 'abu', 'abund', 'abus', 'abusivemen', 'abuyazidmuawiya', 'ac', 'aca', 'acab', 'acaciabrinley', 'acaciamakayla', 'academ', 'academiemm', 'acceler', 'accent', 'accept', 'access', 'accessor', 'accessori', 'acci

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)
clf=tree.DecisionTreeClassifier()
clf.fit(X_train,y_train)
pred_train = clf.predict(X_train)
print('Training accuracy', accuracy_score(y_train, pred_train))

Training accuracy 0.999588864090215


In [12]:
pred_test = clf.predict(X_test)
print('Test accuracy', accuracy_score(y_test, pred_test))



Test accuracy 0.8703946583999046


In [13]:
print(classification_report(y_test,pred_test))

                   precision    recall  f1-score   support

   More offensive       0.49      0.43      0.46       612
Nothing offensive       0.80      0.81      0.80      1410
   less offensive       0.92      0.93      0.92      6365

         accuracy                           0.87      8387
        macro avg       0.73      0.72      0.73      8387
     weighted avg       0.87      0.87      0.87      8387



In [14]:
print(confusion_matrix(y_test, pred_test))

[[ 265   52  295]
 [  42 1142  226]
 [ 238  234 5893]]


In [ ]:
test_data=input("Enter your tweets: ")

df1=cv.transform([test_data]).toarray()
print(clf.predict(df1))